In [1]:
import pandas as pd

In [3]:
from flask import Flask, request, render_template_string
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import threading
from gensim.models import KeyedVectors

from gensim.models import KeyedVectors

filepath = 'GoogleNews-vectors-negative300.bin.gz'
model = KeyedVectors.load_word2vec_format(filepath, binary=True)


In [5]:
df_summarized = pd.read_csv("stage0.csv")

In [6]:
df_summarized.head(5)

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features,combined_text,normalized_text,summary
0,4008728,https://www.airbnb.com/rooms/4008728,20170402075052,2017-04-02,"Luxurious 3 bedroom, centrum, 180m2",‪This luxurious apartment is situated in the c...,In the heart of hip & lively Amsterdam Centrum...,‪This luxurious apartment is situated in the c...,none,There is a wide variety of cafes and restauran...,...,Missing,Amsterdam,strict,1.0,1.52,"52.36523660409065, 4.878249575101092","Host Has Profile Pic,Host Identity Verified,Is...",In the heart of hip & lively Amsterdam Centrum...,in the heart of hip lively amsterdam centrum ...,there is a wide variety of cafes and restauran...
1,7778612,https://www.airbnb.com/rooms/7778612,20170402075052,2017-04-02,Luxury apartment in city centre,"Beautiful, large (105m2, 2 floors) and quiet a...",1 living room 1 dining room 2 bedrooms 1 bathr...,"Beautiful, large (105m2, 2 floors) and quiet a...",none,Within a few minutes walking you'll be in the ...,...,Missing,Amsterdam,strict,1.0,0.81,"52.367309030897516, 4.8738407410699915","Host Is Superhost,Host Has Profile Pic,Host Id...",1 living room 1 dining room 2 bedrooms 1 bathr...,1 living room 1 dining room 2 bedrooms 1 bathr...,1 living room 1 dining room 2 bedrooms 1 bathr...
2,8264596,https://www.airbnb.com/rooms/8264596,20170402075052,2017-04-02,Cosy apartment across Vondelpark,This central located apartment will make you f...,Missing,This central located apartment will make you f...,none,Missing,...,Missing,Amsterdam,flexible,1.0,0.05,"52.361943953804776, 4.866686522141489","Host Has Profile Pic,Host Identity Verified,Is...",Missing This central located apartment will ma...,missing this central located apartment will ma...,missing this central located apartment will ma...
3,2180729,https://www.airbnb.com/rooms/2180729,20170402075052,2017-04-02,Spacious City Apartment Oud-West,Living like an Amsterdam resident in this ligh...,There are 2 bedrooms each with a double bed. I...,Living like an Amsterdam resident in this ligh...,none,The 85 m2 apartment is located on a quiet stre...,...,Missing,Amsterdam,flexible,1.0,0.62,"52.37014616115532, 4.866281767968382","Host Has Profile Pic,Host Identity Verified,Is...",There are 2 bedrooms each with a double bed. I...,there are 2 bedrooms each with a double bed. i...,the spacious living room is situated on the fr...
4,14463171,https://www.airbnb.com/rooms/14463171,20170402075052,2017-04-02,Cosy Studio Apartment Center Amsterdam,Studio Apartment Centre Amsterdam Perfect stud...,Missing,Studio Apartment Centre Amsterdam Perfect stud...,none,Missing,...,Missing,Amsterdam,moderate,1.0,2.04,"52.368817829014425, 4.871249476925455",Host Has Profile Pic,Missing Studio Apartment Centre Amsterdam Perf...,missing studio apartment centre amsterdam perf...,missing studio apartment centre amsterdam perf...


In [7]:
df_dev = df_summarized

In [8]:
#df_dev = df.head(1000)

In [9]:
from simhash import Simhash

def get_simhash_text(text):
    return Simhash(text).value

In [10]:
#df_dev['Name'].fillna('', inplace=True)


In [11]:
df_dev['Title_SimHash'] = df_dev['Name'].apply(get_simhash_text)


In [12]:
objs = [] # a list of (id, simhash)
f = 64



In [13]:
def hamming_distance(hash1, hash2):
    hash1 = int(hash1)
    hash2 = int(hash2)
    x = (hash1 ^ hash2) & ((1 << 64) - 1)
    tot = 0
    while x:
        tot += 1
        x &= x - 1
    return tot

# Threshold for considering items as duplicates
threshold = 12  # Example threshold

duplicates = []
for i in range(len(df_dev)):
    for j in range(i+1, len(df_dev)):
        hash1 = df_dev.iloc[i]['Title_SimHash']
        hash2 = df_dev.iloc[j]['Title_SimHash']
        if hamming_distance(hash1, hash2) <= threshold:
            duplicates.append((i, j))

In [14]:
indexes_to_remove = set([dup[1] for dup in duplicates])

df_dev = df_dev.drop(indexes_to_remove)

df_dev.reset_index(drop=True, inplace=True)

len(df_dev)


958

In [15]:
duplicates

[(17, 54),
 (17, 173),
 (20, 29),
 (33, 67),
 (33, 127),
 (39, 253),
 (44, 173),
 (96, 301),
 (106, 259),
 (106, 328),
 (125, 356),
 (127, 301),
 (129, 392),
 (134, 467),
 (153, 209),
 (158, 200),
 (197, 380),
 (198, 253),
 (218, 301),
 (238, 253),
 (251, 301),
 (253, 576),
 (258, 876),
 (301, 380),
 (310, 462),
 (314, 428),
 (323, 818),
 (328, 426),
 (503, 519),
 (507, 591),
 (539, 685),
 (550, 685),
 (608, 640),
 (608, 642),
 (608, 656),
 (608, 842),
 (631, 650),
 (638, 705),
 (638, 777),
 (638, 854),
 (640, 642),
 (640, 656),
 (640, 842),
 (656, 842),
 (686, 765),
 (751, 774),
 (767, 788),
 (767, 805),
 (774, 869),
 (788, 805),
 (788, 888),
 (792, 861),
 (805, 888),
 (830, 861),
 (837, 873),
 (841, 847),
 (888, 898)]

In [16]:
len(df_dev)

958

In [17]:
df_dev['Beds'] = pd.to_numeric(df_dev['Beds'], errors='coerce').fillna(0).astype(int)

## Keyword Extraction

In [18]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk

def get_keywords(airbnb_description):
# Tokenize and perform part-of-speech tagging
    tokens = word_tokenize(airbnb_description)
    pos_tags = pos_tag(tokens)

# Extract noun phrases using POS tagging
    grammar = r"""
      NP: {<DT>?<JJ.*>*<NN.*>+}
          {<NNP>+}
          {<NN.*>+<IN><NN.*>+}
    """
    cp = nltk.RegexpParser(grammar)
    tree = cp.parse(pos_tags)

    noun_phrases = [' '.join(leaf[0] for leaf in subtree.leaves()) for subtree in tree.subtrees() if subtree.label() == 'NP']
    print("Noun Phrases:", noun_phrases)
    return noun_phrases


In [19]:
df_dev['keywords'] = df_dev['combined_text'].apply(get_keywords)

Noun Phrases: ['the heart', 'hip', 'Amsterdam Centrum/ Oud-West', 'sunny', 'spacious apartment', 'This luxurious apartment', 'the Helmerbuurt neighbourhood', 'a 5-minute walk', 'Leidseplein', 'Vondelpark', 'the Museum area', 'the middle', 'Amsterdam', 'a good night sleep', 'the neighborhood', 'The location', 'the apartment', 'combination', 'the space', 'Amsterdam city centre', 'backyard', 'm2 apartment', 'floors', 'a rooftarrace', 'the lower floor', 'spacious livingroom', 'kitchen', 'a separate toilet', 'the hallway', 'wifi access', 'flat screens tv', 'apple tv', 'The upper floor features', 'beautiful bedrooms', 'an amazing kingsize bed', 'The bathroom', 'a separate shower', 'a great bath', 'shampoo', 'soap', 'the hallway', 'the 2nd separate toilet', 'luxurious apartment', 'the centrum', 'a 5-minute walk', 'Leidseplein', 'The modern apartment', 'floors', 'verdroomd', 'free WIFI', 'A', 'wide variety restaurants', 'cafés', 'the direct surroundings', 'Rijksmuseum', 'walk', 'tramstop', 'me

In [20]:
def compare_keywords(user_input,descrption_keywords,top_n):
    
# Example sentence and list of words
    word_list = descrption_keywords
    tokens_word_list = [word.lower() for word in word_list]

# Tokenize the sentence and word list
    tokens_sentence = get_keywords(user_input)
    
# Compute word embeddings for sentence and word list
    embeddings_sentence = [word2vec_model[word] for word in tokens_sentence if word in word2vec_model.key_to_index]
    embeddings_word_list = [word2vec_model[word] for word in tokens_word_list if word in word2vec_model.key_to_index]
    #print(embeddings_sentence)
    #print(embeddings_word_list)
# Compute cosine similarity between sentence and each word in the list
    #similarity_scores = [cosine_similarity([embeddings_sentence], [embedding_word])[0][0] for embedding_word in embeddings_word_list]
    similarity_matrix = cosine_similarity(embeddings_sentence, embeddings_word_list)
    print(similarity_matrix)
# Sort words based on similarity scores and select top 5
    top_similar_words = [word_list[index] for index in sorted(range(len(similarity_matrix)), key=lambda i: similarity_matrix[i], reverse=True)[:top_n]]
    return top_similar_words

In [21]:
from simhash import Simhash

def get_simhash(word_list):
    result = []
    for word in word_list:
        result.append(Simhash(word).value)
    return result

def hamming_distance(hash1, hash2):
    hash1 = int(hash1)
    hash2 = int(hash2)
    x = (hash1 ^ hash2) & ((1 << 64) - 1)
    tot = 0
    while x:
        tot += 1
        x &= x - 1
    return tot

def get_dup_list(threshold,simhash_list):

    duplicates = []
    for i in range(len(simhash_list)):
        for j in range(i+1, len(simhash_list)):
            #hash1 = df_dev.iloc[i]['Title_SimHash']
            hash1 = simhash_list[i]
            hash2 = simhash_list[j]
            if hamming_distance(hash1, hash2) <= threshold:
                duplicates.append((i, j))
    return duplicates

def get_deduped(duplicates,keywords):
    indexes_to_remove = set([dup[1] for dup in duplicates])
# Create a new list excluding duplicates
    deduplicated_list = [item for index, item in enumerate(keywords) if index not in indexes_to_remove]
    return deduplicated_list

def dedup(keywords):
    threshold = 16
    simhash_list = get_simhash(keywords)
    duplicates = get_dup_list(threshold,simhash_list)
    return get_deduped(duplicates,keywords)

In [22]:
df_dev['keywords_deduped'] = df_dev['keywords'].apply(dedup)  

In [23]:
from nltk.corpus import stopwords
import string
def vectorize_keywords(keywords, model):
    """
    Vectorize a list of keywords using a Word2Vec model by averaging the vectors of the words in each keyword phrase.
    This version removes stopwords and punctuation before vectorization.
    """

    # Load set of English stopwords
    stop_words = set(stopwords.words('english'))
    # Add personal pronouns to be removed
    personal_pronouns = {'you', 'i', 'him', 'her', 'they', 'we', 'us', 'them'}
    stop_words.update(personal_pronouns)
    # Define punctuation characters to be removed
    punctuations = set(string.punctuation)

    keyword_vectors = {}
    for keyword in keywords:
        # Remove stopwords and punctuations, and tokenize
        words = [word for word in keyword.split() if word.lower() not in stop_words and word not in punctuations]
        # Vectorize the remaining words
        word_vectors = [model[word] for word in words if word in model]
        if word_vectors:
            keyword_vector = np.mean(word_vectors, axis=0)
            keyword_vectors[keyword] = keyword_vector
    return keyword_vectors
    
def vectorize_sentence(sentence, model):
    """
    Vectorize a sentence using a Word2Vec model by averaging the vectors of the words in the sentence.
    This version removes stopwords, personal pronouns, and punctuation before vectorization.

    :param sentence: The sentence to be vectorized.
    :param model: A trained Word2Vec model.
    :return: The average vector of the sentence.
    """
    # Load set of English stopwords
    stop_words = set(stopwords.words('english'))
    # Add personal pronouns to be removed
    personal_pronouns = {'you', 'i', 'him', 'her', 'they', 'we', 'us', 'them'}
    stop_words.update(personal_pronouns)
    # Define punctuation characters to be removed
    punctuations = set(string.punctuation)

    # Remove stopwords, personal pronouns, and punctuations, and tokenize
    words = [word for word in sentence.split() if word.lower() not in stop_words and word not in punctuations]
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:
        # If no words in the sentence are in the model, return a zero vector
        return np.zeros(model.vector_size)
    sentence_vector = np.mean(word_vectors, axis=0)
    return sentence_vector
    

def related_keywords(user_input,keywords, model, top_n):
    """
    Returns the top N keywords sorted by their similarity to the vectorized user input.
    """
    # Vectorize user input
    user_input_vector = vectorize_sentence(user_input, model)

    # Vectorize keywords
    keyword_vectors = vectorize_keywords(keywords, model)

    # Calculate similarities and sort
    similarities = {keyword: model.cosine_similarities(user_input_vector, [vector])[0] 
                    for keyword, vector in keyword_vectors.items()}
    sorted_keywords = sorted(similarities, key=similarities.get, reverse=True)
    result = ', '.join(sorted_keywords[:top_n])
    return result


# Word2Vec

In [24]:
df_dev['Beds']

0      3
1      2
2      2
3      4
4      1
      ..
953    3
954    1
955    1
956    1
957    2
Name: Beds, Length: 958, dtype: int64

In [25]:
df_dev_filtered = df_dev[df_dev['Room Type']=='Private room']
df_dev_filtered = df_dev_filtered[df_dev_filtered['Beds']==1]

In [26]:
df_dev_filtered.head()

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Calculated host listings count,Reviews per Month,Geolocation,Features,combined_text,normalized_text,summary,Title_SimHash,keywords,keywords_deduped
7,7969792,https://www.airbnb.com/rooms/7969792,20170402075052,2017-04-02,Lovely room in bright apartment!,Our bright 12m2 guest room with balcony holds ...,Missing,Our bright 12m2 guest room with balcony holds ...,none,"Our neighbourhood, De Baarsjes, is one of the ...",...,1.0,1.1,"52.368868313741864, 4.856909709702364","Host Is Superhost,Host Has Profile Pic",Missing Our bright 12m2 guest room with balcon...,missing our bright 12m2 guest room with balcon...,the area is built in the 1920's in the famous ...,5909911035696876106,"[guest room, balcony, a sleeping couch, comfor...","[guest room, balcony, a sleeping couch, comfor..."
12,17820658,https://www.airbnb.com/rooms/17820658,20170402075052,2017-04-02,Bed & bikes in Oud West Amsterdam,Our loft is located on the 3rd floor. It's 50m...,If you want to discover Amsterdam or if you ju...,Our loft is located on the 3rd floor. It's 50m...,none,Oud West is a vibrant neighborhoud which borde...,...,1.0,Missing,"52.36687672694237, 4.864596414602556","Host Has Profile Pic,Instant Bookable",If you want to discover Amsterdam or if you ju...,if you want to discover amsterdam or if you ju...,there is a nice rooftop terrace where you can ...,2498846536172693372,"[Amsterdam, loft, apartment, some information,...","[Amsterdam, loft, apartment, some information,..."
37,3027232,https://www.airbnb.com/rooms/3027232,20170402075052,2017-04-02,"De Rietzanger 2 , AMSTERDAM",A very cosy room in an just totallt renovated ...,It is a very cosy room with a double bed. A ju...,A very cosy room in an just totallt renovated ...,none,"De rust, het groen, de voorzieningen met zovee...",...,2.0,0.55,"52.36326256551069, 4.865680336225009","Host Has Profile Pic,Is Location Exact",It is a very cosy room with a double bed. A ju...,it is a very cosy room with a double bed. a ju...,superhandig : de tram 1 via leidse plein naar ...,17396742956816064117,"[cosy room, a double bed, new apartment, a lot...","[cosy room, a double bed, new apartment, a lot..."
46,16824179,https://www.airbnb.com/rooms/16824179,20170402075052,2017-04-02,Lovely Room on House Boat,Nice hip room in a cozy neighborhood near City...,"By the stairs you reach the front door, to acc...",Nice hip room in a cozy neighborhood near City...,none,"The neighborhood is lively and cosy, many nice...",...,1.0,2.0,"52.369912959076764, 4.868075967744846","Host Has Profile Pic,Is Location Exact,Instant...","By the stairs you reach the front door, to acc...","by the stairs you reach the front door, to acc...",the room is equipped with a double bed and a s...,15708881586841192538,"[the stairs, the front door, access, the house...","[the stairs, the front door, access, the house..."
48,5973519,https://www.airbnb.com/rooms/5973519,20170402075052,2017-04-02,"Top location, ensuite BR, Roof TeRa",Enjoy a spacious and very quiet bedroom and th...,Do you fancy a perfect city location combined ...,Enjoy a spacious and very quiet bedroom and th...,none,The apartment is on a quiet residential street...,...,3.0,0.83,"52.36582919118046, 4.875321539007233","Host Is Superhost,Host Has Profile Pic,Host Id...",Do you fancy a perfect city location combined ...,do you fancy a perfect city location combined ...,popular restaurants and great breakfastbrunch ...,12266806963334827708,"[a perfect city location, comfortable night sl...","[a perfect city location, comfortable night sl..."


In [27]:
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize


In [28]:
df_dev_filtered['tokenized_summary'] = df_dev_filtered['summary'].apply(word_tokenize)
df_dev['tokenized_summary'] = df_dev['summary'].apply(word_tokenize)



In [29]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to convert sentences to vectors
def sentence_to_vector(sentence, model):
    words = word_tokenize(sentence)
    # Access vectors directly from the model
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)


# Preprocess and vectorize the input sentence
input_sentence = '''I want the place to have a nice rooftop view during the evening, i can bike and walk to explore in the morning'''
input_vector = sentence_to_vector(input_sentence, model)

# Vectorize each description
description_vectors = df_dev_filtered['tokenized_summary'].apply(lambda x: sentence_to_vector(' '.join(x), model))

# Calculate cosine similarity with the input sentence
similarities = [cosine_similarity([input_vector], [desc_vector])[0][0] for desc_vector in description_vectors]

# Combine similarities with descriptions
description_similarity = list(zip( df_dev_filtered['Listing Url'],similarities))

# Sort by similarity
sorted_descriptions = sorted(description_similarity, key=lambda x: x[1], reverse=True)

# Display the most similar descriptions
for desc, similarity in sorted_descriptions[:5]:  # Adjust the number to display as needed
    print(f"Description: {desc}\nSimilarity: {similarity}\n")


#Description: tram 10 goes all the way around the city center, bringing you to the leidseplein, museum square, the jordaan the east of amsterdam is developing rapidly with more and more interesting places and nice bars and restaurants. walking distance to the center, easy access to public transport and restaurants, pubs and shops around the corner. tram 26 and tram 10 stop around the corner rietlandpark tram 26 brings you in 5 minutes to the central station. tram 26 and tram 10 stop around the corner rietlandpark tram 26 brings you in 5 minutes to the central station. on the same floor there are the bathroom with shower and bath and the toilet.



Description: https://www.airbnb.com/rooms/17820658
Similarity: 0.8606014847755432

Description: https://www.airbnb.com/rooms/13869512
Similarity: 0.8582753539085388

Description: https://www.airbnb.com/rooms/14226639
Similarity: 0.8573563098907471

Description: https://www.airbnb.com/rooms/13387863
Similarity: 0.8550234436988831

Description: https://www.airbnb.com/rooms/13449508
Similarity: 0.8513443470001221



In [30]:
df_dev_filtered['Similarity'] = similarities
df_dev_filtered['Input Sentence'] = input_sentence

# Select relevant columns and sort by similarity
result_df = df_dev_filtered[['Name', 'Listing Url', 'Similarity', 'normalized_text', 'summary']].sort_values(by='Similarity', ascending=False)

# Display the DataFrame
print(result_df.head())

                                      Name  \
12       Bed & bikes in Oud West Amsterdam   
378     sunny panoramic loft in citycenter   
336                          Plaka Village   
613   Cozy room 5 mins from Morden Station   
503  Cosy, homey, a walk to Brixton Market   

                               Listing Url  Similarity  \
12   https://www.airbnb.com/rooms/17820658    0.860601   
378  https://www.airbnb.com/rooms/13869512    0.858275   
336  https://www.airbnb.com/rooms/14226639    0.857356   
613  https://www.airbnb.com/rooms/13387863    0.855023   
503  https://www.airbnb.com/rooms/13449508    0.851344   

                                       normalized_text  \
12   if you want to discover amsterdam or if you ju...   
378  the loft is airy and roomy, and very stylish i...   
336  this is a unique but quite small room. for thi...   
613  the room is in our cosy maisonette, just a sho...   
503  missing victoria and northern line. 25 min to ...   

                         

In [31]:
result_df.head()

,Name,Listing Url,Similarity,normalized_text,summary
12,Bed & bikes in Oud West Amsterdam,https://www.airbnb.com/rooms/17820658,0.860601,if you want to discover amsterdam or if you ju...,there is a nice rooftop terrace where you can ...
378,sunny panoramic loft in citycenter,https://www.airbnb.com/rooms/13869512,0.858275,"the loft is airy and roomy, and very stylish i...","connect you to many areas of amsterdam, and yo..."
336,Plaka Village,https://www.airbnb.com/rooms/14226639,0.857356,this is a unique but quite small room. for thi...,you will enjoy the lovely balcony with its cla...
613,Cozy room 5 mins from Morden Station,https://www.airbnb.com/rooms/13387863,0.855023,"the room is in our cosy maisonette, just a sho...",the room is a short five minute walk to both m...
503,"Cosy, homey, a walk to Brixton Market",https://www.airbnb.com/rooms/13449508,0.851344,missing victoria and northern line. 25 min to ...,you will love my place because of the cosiness...


In [32]:
def sentence_to_vector(sentence, model):
    words = word_tokenize(sentence)
    word_vectors = [model.get_vector(word) for word in words if word in model.key_to_index]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)


def search_dataset1(input_sentence, filtered_data):
    input_vector = sentence_to_vector(input_sentence, model)

    # Ensure that 'tokenized_summary' is updated based on the filtered data
    filtered_data['tokenized_summary'] = filtered_data['summary'].apply(word_tokenize)

    # Vectorize each description in the filtered data
    description_vectors = filtered_data['tokenized_summary'].apply(lambda x: sentence_to_vector(' '.join(x), model))

    # Calculate cosine similarity with the input sentence
    similarities = [cosine_similarity([input_vector], [desc_vector])[0][0] for desc_vector in description_vectors]

    # Combine similarities with descriptions from the filtered data
    description_similarity = list(zip(filtered_data['Name'], filtered_data['Listing Url'], similarities))

    # Sort by similarity
    sorted_descriptions = sorted(description_similarity, key=lambda x: x[2], reverse=True)

    return sorted_descriptions[:5]  # Return top 5 results


In [293]:
def search_dataset(input_sentence, filtered_data):
    input_vector = sentence_to_vector(input_sentence, model)

    #Ensure that 'tokenized_summary' is updated based on the filtered data
    filtered_data['tokenized_summary'] = filtered_data['summary'].apply(word_tokenize)

    # Vectorize each description in the filtered data
    description_vectors = filtered_data['tokenized_summary'].apply(lambda x: sentence_to_vector(' '.join(x), model))


    # Calculate cosine similarity with the input sentence
    similarities = [cosine_similarity([input_vector], [desc_vector])[0][0] for desc_vector in description_vectors]

    filtered_data['matching_keywords'] = filtered_data['keywords_deduped'].apply(lambda x: related_keywords(input_sentence, x , model, 7))

    # Assigning the split keywords to a new column 'split_keywords'
    filtered_data['split_keywords'] = filtered_data['matching_keywords'].apply(lambda x: x.split(','))

    print("FOR TEXT:")
    print(filtered_data['split_keywords'][0])
    
    # Combine similarities with descriptions from the filtered data
    description_similarity = list(zip(filtered_data['Name'], filtered_data['Listing Url'],filtered_data['split_keywords'], similarities))

    # Sort by similarity
    sorted_descriptions = sorted(description_similarity, key=lambda x: x[3], reverse=True)

    sorted_descriptions[:5]

 


    return sorted_descriptions[:5]  # Return top 5 results


# Flask

app = Flask(__name__)
html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Property Search</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f0f0f0;
        }
        .container {
            width: 80%;
            margin: auto;
            overflow: hidden;
            padding: 20px;
        }
        form {
            background: #fff;
            padding: 20px;
            margin-top: 20px;
        }
        label, select, input[type="text"] {
            margin-top: 10px;
            display: block;
            width: 100%;
        }
        input[type="submit"] {
            background: #333;
            color: #fff;
            border: none;
            padding: 10px;
            cursor: pointer;
            margin-top: 10px;
        }
        input[type="submit"]:hover {
            background: #444;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Search Properties</h1>
        <form method="post">
            <label for="beds">Beds:</label>
            <select name="beds" id="beds">
                <option value="">Select Beds</option>
                {% for bed in unique_beds %}
                    <option value="{{ bed }}">{{ bed }}</option>
                {% endfor %}
            </select>

            <label for="room_type">Room Type:</label>
            <select name="room_type" id="room_type">
                <option value="">Select Room Type</option>
                {% for room_type in unique_room_types %}
                     <option value="{{ room_type }}">{{ room_type }}</option>
                {% endfor %}
            </select>

            <label for="country">Country:</label>
            <select name="country" id="country">
                <option value="">Select Country</option>
                {% for country in unique_countries %}
                    <option value="{{ country }}">{{ country }}</option>
                {% endfor %}
            </select>

            <label for="city">City:</label>
            <select name="city" id="city">
                <option value="">Select City</option>
                {% for city in unique_cities %}
                    <option value="{{ city }}">{{ city }}</option>
                {% endfor %}
            </select>

            <label for="input_sentence">Enter a Sentence:</label>
            <input type="text" id="input_sentence" name="input_sentence">

            <input type="submit" value="Search">
        </form>

        {% if results %}
            <h2>Results:</h2>
            <ul>
                {% for name, url, similarity in results %}
                    <li>
                        <strong>Name:</strong> {{ name }} | 
                        <strong>URL:</strong> <a href="{{ url }}" target="_blank">{{ url }}</a> | 
                        <strong>Similarity:</strong> {{ similarity }}
                    </li>
                {% endfor %}
            </ul>
        {% else %}
            <p>No results to display.</p>
        {% endif %}
    </div>
</body>
</html>

"""

In [123]:
search_html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Property Search</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f0f0f0;
        }
        .container {
            width: 80%;
            margin: auto;
            overflow: hidden;
            padding: 20px;
        }
        form {
            background: #fff;
            padding: 20px;
            margin-top: 20px;
        }
        label, select, input[type="text"] {
            margin-top: 10px;
            display: block;
            width: 100%;
        }
        input[type="submit"] {
            background: #333;
            color: #fff;
            border: none;
            padding: 10px;
            cursor: pointer;
            margin-top: 10px;
        }
        input[type="submit"]:hover {
            background: #444;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Search Properties</h1>
        <form action="{{ url_for('results') }}" method="get">
            <label for="beds">Beds:</label>
            <select name="beds" id="beds">
                <option value="">Select Beds</option>
                {% for bed in unique_beds %}
                    <option value="{{ bed }}">{{ bed }}</option>
                {% endfor %}
            </select>

            <label for="room_type">Room Type:</label>
            <select name="room_type" id="room_type">
                <option value="">Select Room Type</option>
                {% for room_type in unique_room_types %}
                    <option value="{{ room_type }}">{{ room_type }}</option>
                {% endfor %}
            </select>

            <label for="country">Country:</label>
            <select name="country" id="country">
                <option value="">Select Country</option>
                {% for country in unique_countries %}
                    <option value="{{ country }}">{{ country }}</option>
                {% endfor %}
            </select>

            <label for="city">City:</label>
            <select name="city" id="city">
                <option value="">Select City</option>
                {% for city in unique_cities %}
                    <option value="{{ city }}">{{ city }}</option>
                {% endfor %}
            </select>

            <label for="input_sentence">Enter a Sentence:</label>
            <input type="text" id="input_sentence" name="input_sentence">

            <input type="submit" value="Search">
        </form>
    </div>
</body>
</html>
"""


In [190]:
results_html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Search Results</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f0f0f0;
        }
        .container {
            width: 80%;
            margin: auto;
            overflow: hidden;
            padding: 20px;
        }
        ul {
            list-style-type: none;
            padding: 0;
        }
        li {
            background: #fff;
            padding: 10px;
            margin-top: 10px;
        }
        a {
            color: #00f;
        }
        a:hover {
            text-decoration: underline;
        }
        .back-link {
            margin-top: 20px;
            display: block;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Search Results</h1>
        {% if results %}
            <ul>
                {% for name, url, keywords, similarity in results %}
                    <li>
                        <strong>Name:</strong> {{ name }}<br>
                        <strong>Property Features:</strong> {{ keywords }}<br>
                        <strong>URL:</strong> <a href="{{ url }}" target="_blank">{{ url }}</a><br>
                        <strong>Similarity:</strong> {{ similarity }}
                    </li>
                {% endfor %}
            </ul>
        {% else %}
            <p>No results to display.</p>
        {% endif %}
        <a href="{{ url_for('index') }}" class="back-link">Back to Search</a>
    </div>
</body>
</html>
"""


In [125]:
unique_beds = sorted(df_dev['Beds'].unique())
unique_room_types = sorted(df_dev['Room Type'].dropna().unique())
unique_countries = sorted(df_dev['Country'].dropna().unique())
unique_cities = sorted(df_dev['City'].dropna().unique())


In [126]:
unique_beds

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 16]

In [369]:
from flask import Flask, render_template

app = Flask(__name__)


In [370]:
@app.route('/')
def index():
    # Render the search form
    return render_template('index3.html', 
                                  unique_beds=unique_beds, 
                                  unique_room_types=unique_room_types, 
                                  unique_countries=unique_countries, 
                                  unique_cities=unique_cities)

@app.route('/results')
def results():
    selected_beds = request.args.get('beds')
    selected_room_type = request.args.get('room_type')
    selected_country = request.args.get('country')
    selected_city = request.args.get('city')
    input_sentence = request.args.get('input_sentence', '')
    
    # Filter the dataset based on the selections
    filtered_data = df_dev
    if selected_beds:
        try:
            selected_beds_int = int(selected_beds)
            filtered_data = filtered_data[filtered_data['Beds'] == selected_beds_int]
        except ValueError:
            # Handle invalid input
            print(f"Invalid input for beds: {selected_beds}")
    if selected_room_type:
        filtered_data = filtered_data[filtered_data['Room Type'] == selected_room_type]
    if selected_country:
        filtered_data = filtered_data[filtered_data['Country'] == selected_country]
    if selected_city:
        filtered_data = filtered_data[filtered_data['City'] == selected_city]

    search_results = None
    if input_sentence:
        search_results = search_dataset(input_sentence, filtered_data)

    # Render the results page
    return render_template('index4.html', results=search_results)
    return render_template_string(results_html_template, results=search_results)


@app.route('/', methods=['GET', 'POST'])
def index():
    results = None
    if request.method == 'POST':
        selected_beds = request.form.get('beds')
        selected_room_type = request.form.get('room_type')
        selected_country = request.form.get('country')
        selected_city = request.form.get('city')
        input_sentence = request.form.get('input_sentence', '')

        # Filter the dataset based on the selections
        filtered_data = df_dev
        if selected_beds:
            try:
                selected_beds_int = int(selected_beds)
                filtered_data = filtered_data[filtered_data['Beds'] == selected_beds_int]
            except ValueError:
                # Handle invalid input
                print(f"Invalid input for beds: {selected_beds}")

        if selected_room_type:
            filtered_data = filtered_data[filtered_data['Room Type'] == selected_room_type]
        if selected_country:
            filtered_data = filtered_data[filtered_data['Country'] == selected_country]
        if selected_city:
            filtered_data = filtered_data[filtered_data['City'] == selected_city]

        # Process the filtered data and input sentence
        if input_sentence:
            results = search_dataset(input_sentence, filtered_data)

    return render_template_string(html_template, results=results, unique_beds=unique_beds, 
                                  unique_room_types=unique_room_types, unique_countries=unique_countries, 
                                  unique_cities=unique_cities)




In [ ]:
import threading

# Function to run the Flask app in a separate thread
def run_app_in_thread():
    # This allows the Flask app to be stopped when the Jupyter kernel is interrupted
    app.run(port=5088, use_reloader=False, threaded=True)

# Create a thread for running the app
app_thread = threading.Thread(target=run_app_in_thread)

# Start the Flask app in the background thread
app_thread.start()

# Reminder for the user
"Flask app is running! Access it at http://127.0.0.1:5001/ in your web browser. To stop the app, interrupt the Jupyter kernel."

 * Serving Flask app '__main__' (lazy loading)


'Flask app is running! Access it at http://127.0.0.1:5001/ in your web browser. To stop the app, interrupt the Jupyter kernel.'

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5088 (Press CTRL+C to quit)
127.0.0.1 - - [05/Dec/2023 00:39:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 00:39:13] "GET /static/images/bg1.webp HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 00:39:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/Dec/2023 00:39:26] "GET /results?input_sentence=I+want+the+place+to+have+a+nice+rooftop+view+during+the+evening,+i+can+bike+and+walk+to+explore+in+the+morning&country=Netherlands&city=Amsterdam&beds=&room_type= HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2023 00:39:26] "GET /static/images/fallback2.jpg HTTP/1.1" 200 -


FOR TEXT:
['a 5-minute walk', ' walk', ' a bike', ' the bike', ' view', ' an amazing kingsize bed', ' fun']


In [41]:
result_df = result_df.head(6)

In [42]:
result_df

,Name,Listing Url,Similarity,normalized_text,summary
12,Bed & bikes in Oud West Amsterdam,https://www.airbnb.com/rooms/17820658,0.860601,if you want to discover amsterdam or if you ju...,there is a nice rooftop terrace where you can ...
378,sunny panoramic loft in citycenter,https://www.airbnb.com/rooms/13869512,0.858275,"the loft is airy and roomy, and very stylish i...","connect you to many areas of amsterdam, and yo..."
336,Plaka Village,https://www.airbnb.com/rooms/14226639,0.857356,this is a unique but quite small room. for thi...,you will enjoy the lovely balcony with its cla...
613,Cozy room 5 mins from Morden Station,https://www.airbnb.com/rooms/13387863,0.855023,"the room is in our cosy maisonette, just a sho...",the room is a short five minute walk to both m...
503,"Cosy, homey, a walk to Brixton Market",https://www.airbnb.com/rooms/13449508,0.851344,missing victoria and northern line. 25 min to ...,you will love my place because of the cosiness...
508,Quirky cosy mid century design home,https://www.airbnb.com/rooms/8768118,0.848542,our flat is over 100 years old on a conservati...,at the top of the victoria line only 25 mins f...


127.0.0.1 - - [03/Dec/2023 23:49:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2023 23:49:42] "GET /favicon.ico HTTP/1.1" 404 -
